In [1]:
import pandas as pd
# 과학적 표기 안하도록
pd.options.display.float_format = '{:.2f}'.format

import numpy as np
import datetime

---
### 증감 결과 확인을 위한 함수

In [2]:
def increasing(col):
    for i in range(len(col)):
        if(i == 0):
            temp = col[i]
        else:
            if(temp > col[i]):
                return 0
            temp = col[i]
    return 1 

def decreasing(col):
    for i in range(len(col)):
        if(i==0):
            temp = col[i]
        else:
            if(temp < col[i]):
                return 0
            temp = col[i]
    return 1

---
### request+bs로 크롤링


In [3]:
import requests
from bs4 import BeautifulSoup as bs

In [32]:
# code로 접근 가능한 url
code_url = "https://finance.naver.com/item/main.nhn?code="
# 종목 분석 url
coinfo_url = "https://finance.naver.com/item/coinfo.nhn?code="
# 종목 분석 url (!naver)
coinfo_url_2 = "https://navercomp.wisereport.co.kr/v2/company/c1050001.aspx?cmp_cd=%s&cn="
# 시가총액 목록 url
size_market_url_0 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page="

size_market_url_1 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page="

In [33]:
# 오늘의 시가총액 상위 400 code 목록 구하기

codeL = []
# 코스피 상위 300
for i in range(1,7):
    soup = bs(requests.get(size_market_url_0+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL
# 코스닥 상위 100
for i in range(1, 3):
    soup = bs(requests.get(size_market_url_1+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL
             

In [42]:
# soup = bs(requests.get(size_market_url_0+str(2)).text, 'html.parser')
# tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
# soup.select("table ")[1].select_one("tbody").select("a.tltle")

In [43]:
# 오늘의 시가총액 code 목록 저장 
todayDate = datetime.datetime.now().strftime("%Y_%b_%d")
f = open("./"+todayDate+"_시가총액_상위권.txt", 'w')
f.write("\n".join(codeL))
f.close()

# 오늘의 시가총액 code 목록 불러오기
useDate = "2020_Feb_18"
f = open("./"+useDate+"_시가총액_상위권.txt", "r")
codeL = [x.replace("\n", "") for x in f.readlines()]
f.close()

---

### 컨센서스 확인 함수

In [44]:
# 종목 이름 얻기
code = "005387"
soup = bs(requests.get(code_url + code).text, 'html.parser')
name = soup.select_one("#middle > div.h_company > div.wrap_company > h2 > a").text
name

'현대차2우B'

In [56]:
def Consensus(code, score25, score23, score20):
    code = str(code)
        
    # 종목 이름 얻기
    soup = bs(requests.get(code_url + code).text, 'html.parser')
    name = soup.select_one("#middle > div.h_company > div.wrap_company > h2 > a").text
    
    # 컨센서스 테이블 구축
    consenUrl = "https://navercomp.wisereport.co.kr/company/ajax/c1050001_data.aspx?flag=2&cmp_cd="\
                + code + "&finGubun=MAIN&frq=0&chartType=svg"
    
    # 컨센서스 없는 경우 1
    if(requests.get(consenUrl).json()["JsonData"] == []):
        return 
    df_table = pd.DataFrame(requests.get(consenUrl).json()["JsonData"]).iloc[:, :-1]

    # 매출액 ~ EV/EBITDA 숫자형으로 변경
    df_table.iloc[:, 1:-1] = df_table.iloc[:,1:-1].applymap(lambda x: "0" if x == "" else x).applymap(lambda x: float(x.replace(",","")))
    
    # 컨센서스 없는 경우 2
    if(df_table.iloc[:, 1:-1].sum().sum() == 0):
        return
    
    # 증감 결과 df table로 확인
    result_row = [0]*12
    for inc in [1, 2, 3, 4, 5, 6, 9]:
        if(increasing(df_table.iloc[:, inc])):
            result_row[inc] = 1
    for dec in [7, 8, 10]:
        if(decreasing(df_table.iloc[:, dec])):
            result_row[dec] = 1

#     result_df = pd.DataFrame([result_row], columns = df_table.columns)
#     df_table.append(result_df, ignore_index = True)
    
    # 증감 결과 매출액 / 영업이익 / per * 10 roe * 5
    formula = [0,10,1,10,1,1,1,10,1,5,1,0]
    score = np.dot(result_row,formula)

    # 점수 결과
    return [code, name, str(score)]


---

## 시가총액 상위 400개의 컨센서스 확인

In [57]:
score25 = []
score23 = [] 
score20 = []
for i in range(400):
    if(i % 100 == 99):
        print(i)
    
    result = Consensus(codeL[i], score25, score23, score20)
    if(result):
        if(int(result[2]) >= 25):
            score25.append(" ".join(result))
        elif(int(result[2]) >= 23):
            score23.append(" ".join(result))
        elif(int(result[2]) >= 20):
            score20.append(" ".join(result))
    

99
199
299
399


In [58]:
score25

['280360 롯데제과 27',
 '336370 두산솔루스 28',
 '005180 빙그레 28',
 '336260 두산퓨얼셀 29',
 '033290 코웰패션 33']

In [59]:
score23

['051900 LG생활건강 23',
 '018260 삼성에스디에스 23',
 '316140 우리금융지주 24',
 '271560 오리온 23',
 '012510 더존비즈온 23',
 '284740 쿠쿠홈시스 23',
 '298020 효성티앤씨 23',
 '003230 삼양식품 23',
 '058470 리노공업 23',
 '030190 NICE평가정보 23',
 '035900 JYP Ent. 23',
 '064760 티씨케이 23',
 '200130 콜마비앤에이치 23',
 '067160 아프리카TV 23',
 '060250 NHN한국사이버결제 23',
 '267980 매일유업 23',
 '272290 이녹스첨단소재 23']

In [60]:
score20

['000120 CJ대한통운 21',
 '282330 BGF리테일 21',
 '001040 CJ 21',
 '030000 제일기획 22',
 '007310 오뚜기 21',
 '181710 NHN 21',
 '009420 한올바이오파마 21',
 '192080 더블유게임즈 21',
 '286940 롯데정보통신 21',
 '145720 덴티움 21',
 '298050 효성첨단소재 21',
 '253450 스튜디오드래곤 21',
 '036830 솔브레인 21',
 '086450 동국제약 21',
 '044340 위닉스 22',
 '230240 에치에프알 21']

---

#### 오늘의 점수 저장

In [63]:
score25_code = [x.split()[0] for x in score25]
score23_code = [x.split()[0] for x in score23]
score20_code = [x.split()[0] for x in score20]

In [64]:
# 오늘의 socre 저장 
todayDate = datetime.datetime.now().strftime("%Y_%b_%d")
f = open("./scoreData/"+todayDate+"_SCORE>=25.txt", 'w')
f.write("\n".join(score25_code))
f.close()

todayDate = datetime.datetime.now().strftime("%Y_%b_%d")
f = open("./scoreData/"+todayDate+"_SCORE>=23.txt", 'w')
f.write("\n".join(score23_code))
f.close()

todayDate = datetime.datetime.now().strftime("%Y_%b_%d")
f = open("./scoreData/"+todayDate+"_SCORE>=20.txt", 'w')
f.write("\n".join(score20_code))
f.close()

---

## 나의 네이버증권 셋팅하기

In [66]:
from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome('./chromedriver')

In [67]:
itemListUrl = "https://finance.naver.com/mystock/itemList.nhn?groupId=6&type=OVERALL"

In [68]:
# 기존 관심종목 삭제
# for i in range(6, 9):

# 링크 접속 및 로그인
driver.get(itemListUrl) # 로그인 필요 가능성

In [113]:
# 기존 관심 종목 삭제
driver.get(itemListUrl)
driver.find_element_by_css_selector("#all_select").click()
driver.find_element_by_css_selector("#wrap > div.section_mys > div.group_mystb.NE\=a\:lst > div:nth-child(2) > button.btn_del.NPI\=a\:del").click()
Alert(driver).accept()


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=58696): Max retries exceeded with url: /session/799e7b3932d6a7659936b6d0b43103c1/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x123eb9eb8>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [112]:
# 관심종목 목록 등록
for nowCode in score25_code:
    driver.implicitly_wait(500)
    try:
        driver.find_element_by_css_selector("#mystock_input").clear()
    except:
        print("e")
    driver.implicitly_wait(500)
    driver.find_element_by_css_selector("#mystock_input").send_keys(nowCode)
    driver.implicitly_wait(5)
    driver.find_element_by_css_selector("#mystock_input").send_keys(Keys.RETURN)

e


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=58696): Max retries exceeded with url: /session/799e7b3932d6a7659936b6d0b43103c1/timeouts (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x123fd5550>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [97]:
score25_code


['280360', '336370', '005180', '336260', '033290']

In [103]:
nowCode = "336370"
driver.find_element_by_css_selector("#mystock_input").clear()
driver.find_element_by_css_selector("#mystock_input").send_keys(nowCode)
driver.find_element_by_css_selector("#mystock_input").send_keys(Keys.RETURN)